In [1]:
import selenium
from selenium import webdriver
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as ec
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.common.keys import Keys
from bs4 import BeautifulSoup
from selenium.webdriver.common.by import By
import time
from datetime import datetime

Data Scraped:
1. Honors and Awards
2. Languages
3. Skills
4. Certs
5. Educations
6. Experiences
7. Connections, Followers
8. About
9. Contact details

# Login

In [2]:
driver = webdriver.Edge()
driver.get('https://www.linkedin.com/login')

def login(username, pw):    
    driver.find_element(By.ID, "username").send_keys(username)
    driver.find_element(By.ID, "password").send_keys(pw)
    driver.find_element(By.CLASS_NAME, "login__form_action_container").click()

actions = ActionChains(driver)
login('shreyashthengne@gmail.com', 'Qwerasdf@2003&')

# Searching

In [4]:
def search(query):
    driver.find_element(By.ID, 'global-nav-search').click()
    actions.send_keys(query).send_keys(Keys.ENTER).perform()
    WebDriverWait(driver, 10).until(ec.presence_of_all_elements_located((By.CLASS_NAME, "search-reusables__primary-filter")))[0].click()
    actions.send_keys(Keys.ESCAPE).perform()

search('amazon sde')

In [5]:
def create_soup(source_code):
    return BeautifulSoup(source_code, 'html.parser')

In [6]:
pages = 4

In [32]:
def link_scraper():
    profile_links = []
    for k in range(pages):
        soup = create_soup(driver.page_source)

        for i in soup.find_all('li', {'class':"reusable-search__result-container"}):
            try:
                profile_links.append(i.find('a',{'class':"app-aware-link scale-down"}).get('href'))
            except:
                print('Ad Encountered')

        print(f"Page {k + 1}")
    # try:
        actions.send_keys(Keys.END)
        time.sleep(2)
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        WebDriverWait(driver, 10).until(ec.presence_of_element_located((By.CLASS_NAME, "artdeco-pagination__button--next"))).click()
        # except:
        #     return link_scraper()
    return profile_links

In [40]:
profiles = link_scraper()

Ad Encountered
Page 1
Page 2
Page 3
Page 4


In [41]:
driver.get(profiles[0])

In [42]:
profile_link = driver.current_url

In [43]:
profile_link

'https://www.linkedin.com/in/abhishek-paithane-a617a3119/'

Getting the soup ready

In [54]:
# this is the collection of all the cards on linkedin profile (like about section, activity section, experience, etc)
soup = create_soup(driver.page_source)
cards = soup.find_all('section', {"class": "artdeco-card pv-profile-card break-words mt2"})

In [64]:
def get_contact_data():
    try:
        WebDriverWait(driver,10)
        intro_card = soup.find('section', {'class':'artdeco-card TIIKQjPaJzLNuejCihgbgRkECKXApRs'})
        name = intro_card.find('a', {'class':'ember-view pv-text-details__about-this-profile-entrypoint'}).text.strip()

        driver.find_element(By.ID, "top-card-text-details-contact-info").click()
        obj_list = WebDriverWait(driver, 10).until(ec.presence_of_all_elements_located((By.CLASS_NAME, "pv-contact-info__contact-type")))
        contact_info = []
        for sel_obj in obj_list:
            contact_info.append(sel_obj.text.split("\n"))
        try:
            contact_info.remove(['Birthday'])
        except:
            pass

        def nested_list_to_dict(l):
            d = {}
            for i in l:
                d[i[0]] = i[1]
            return d

        actions = ActionChains(driver)
        actions.send_keys(Keys.ESCAPE).perform()
        contact_info_dict = nested_list_to_dict(contact_info)
        contact_info_dict['name'] = name

        if contact_info_dict == []:
            contact_info_dict = get_contact_data()
        
        contact_info_dict['profile_link'] = contact_info_dict[contact_info_dict['name'].split(" ")[0] + "’s " + 'Profile']
        contact_info_dict.pop(contact_info_dict['name'].split(" ")[0] + "’s " + 'Profile')
            
        for field in ['Website', 'Email', 'IM','Birthday','Connected', 'Address', 'Phone', 'name', 'profile_link']:
            if field not in contact_info_dict:
                contact_info_dict[field] = None
        return contact_info_dict
    
    except:
        driver.refresh()
        return get_contact_data()
    
contacts = get_contact_data()

In [65]:
sections_available = []
for i in cards:
    sections_available.append(i.find().get('id'))
# sections_available

# Connections, followers, about

In [98]:
connections = soup.find('div', {'class', "mt2 relative"}).findNextSibling().text.strip()
for i in cards:
    if i.find('div', {'id':'content_collections'}) != None:
        followers = i.find('div', {'class':'pvs-header__title-container'}).text.strip().split()[1]
        break


for i in cards:
    if i.find('div', {'id':'about'}) != None:
        about = i.find('div', {'class':"display-flex ph5 pv3"}).text.strip()
        break

about = about[:len(about)//2]

# Experiences

In [102]:
def get_experience():
    try:
        driver.get(profile_link + 'details/experience/')
        WebDriverWait(driver, 10)
        soup = create_soup(driver.page_source)

        all_exp = soup.find_all('li', {'class', 'pvs-list__paged-list-item artdeco-list__item pvs-list__item--line-separated pvs-list__item--one-column'})

        def extract_exp_data(sample):
            '''
            role, company, job_type, from, to, duration
            '''
            
            role = sample.find('div', {'class', 'display-flex flex-wrap align-items-center full-height'}).text.strip()
            role = role[:len(role)//2]

            company_and_job_type = sample.find('span', {'class', 't-14 t-normal'}).text.strip()
            company_and_job_type = company_and_job_type[:len(company_and_job_type)//2].split(" · ")
            company = company_and_job_type[0]
            
            try:
                job_type = company_and_job_type[1]
            except:
                job_type = None

            period, duration = sample.find('span', {'class', "pvs-entity__caption-wrapper"}).text.split(' · ')
            period = period.split(" - ")
            
            try:
                duration = int(duration.split(" ")[0]) * 12 + int(duration.split(" ")[2])
            except:
                duration = int(duration.split(" ")[0])

            try:
                period = {'from': datetime.strptime(period[0], "%b %Y"), 'to': datetime.strptime(period[1], "%b %Y")}
            except:
                period = {'from': datetime.strptime(period[0], "%b %Y"), 'to': datetime.strptime(datetime.now().strftime("%b %Y"), "%b %Y")}
            return {'role':role, 'company':company, 'job_type':job_type, 'from':period['from'], 'to':period['to'], 'duration':duration}

        all_experiences = []
        for i in  all_exp:
            all_experiences.append(extract_exp_data(i))

        if all_experiences == []:
            all_experiences = get_experience()

        return all_experiences
    except:
        return get_experience()
if 'experience' in sections_available:
    experiences = get_experience()
else:
    experiences = [{
        'role':None,
        'company': None,
        'job_type': None,
        'from': None,
        'to': None,
        'duration': None
    }]

# Educations

In [103]:
def get_education():
    try:
        driver.get(profile_link + 'details/education/')
        WebDriverWait(driver, 10)
        soup = create_soup(driver.page_source)

        all_edu = soup.find_all('li', {'class', 'pvs-list__paged-list-item artdeco-list__item pvs-list__item--line-separated pvs-list__item--one-column'})

        def extract_education_data(sample):
            '''
            college, degree, period
            '''
            college = sample.find('div', {'class', 'display-flex flex-wrap align-items-center full-height'}).text.strip()
            college = college[:len(college)//2]
            degree = sample.find('span', {'class', 't-14 t-normal'}).text.strip()
            degree = degree[:len(degree)//2]
            period = sample.find('span', {'class', 'pvs-entity__caption-wrapper'}).text.strip().split(" - ")
            period = {'from': period[0], 'to': period[1]}
            return {'college':college, 'degree':degree, 'from':period['from'], 'to':period['to']}

        all_education = []
        for i in  all_edu:
            all_education.append(extract_education_data(i))
        
        if all_education == []:
            all_education = get_education()
        
        return all_education
    except:
        return get_education()
if 'education' in sections_available:
    education = get_education()
else:
    education = [{
        'college': None,
        'degree': None,
        'from': None,
        'to': None
    }]

# Certifications

In [104]:
def get_certifications():
    try:
        driver.get(profile_link + 'details/certifications/')
        WebDriverWait(driver, 10)
        soup = create_soup(driver.page_source)

        def extract_certifications(sample):
            '''
            name, org, issued, link
            '''
            sample = all_certs[0]
            name = sample.find('div', {'class', 'display-flex flex-wrap align-items-center full-height'}).text.strip()
            organization = sample.find('span', {'class', 't-14 t-normal'}).text
            organization = organization[:len(organization)//2]
            issued = datetime.strptime(sample.find('span', {'class', 'pvs-entity__caption-wrapper'}).text.split("Issued")[1].strip(), '%b %Y')
            link = sample.find('a', {'class', 'optional-action-target-wrapper artdeco-button artdeco-button--secondary artdeco-button--standard artdeco-button--2 artdeco-button--muted inline-flex justify-center align-self-flex-start button-placement-wrap'}).get('href')
            return {'cert_name':name, 'cert_org': organization, 'cert_issued':issued, 'cert_link':link}

        all_certs = soup.find_all('li', {'class', 'pvs-list__paged-list-item artdeco-list__item pvs-list__item--line-separated pvs-list__item--one-column'})

        all_certification = []
        for i in  all_certs:
            all_certification.append(extract_certifications(i))

        if all_certification == []:
            all_certification = get_certifications()

        return all_certification
    except:
        return get_certifications()

if 'licenses_and_certifications' in sections_available:
    certifications = get_certifications()
else:
    certifications = [{
        'cert_name': None,
        'cert_org': None,
        'cert_issued': None,
        'cert_link': None
  }]

# Skills

In [105]:
def get_skills():
    try:
        driver.get(profile_link + 'details/skills/')
        WebDriverWait(driver, 10)
        soup = create_soup(driver.page_source)

        all_skills = soup.find_all('li', {'class', 'pvs-list__paged-list-item artdeco-list__item pvs-list__item--line-separated pvs-list__item--one-column'})

        def extract_skills(sample):
            '''
            skill
            '''
            skill = sample.find('div', {'class':"display-flex flex-wrap align-items-center full-height"}).text.strip()
            skill = skill[:len(skill)//2]
            return skill

        skills = set()
        for i in all_skills:
            skills.add(extract_skills(i))
        skills = list(skills)

        if skills == []:
            skills = get_skills()
        
        return skills
    except:
        return get_skills()

if 'skills' in sections_available:
    skills = get_skills()
else:
    skills = [None]

# Languages

In [106]:
def get_languages():
    try:
        driver.get(profile_link + 'details/languages/')
        wait = WebDriverWait(driver, 10)
        soup = create_soup(driver.page_source)

        all_langs = soup.find_all('li', {'class', 'pvs-list__paged-list-item artdeco-list__item pvs-list__item--line-separated pvs-list__item--one-column'})

        def extract_langs(sample):
            '''
            languages, profiency
            '''
            lang = sample.find('div', {'class':"display-flex flex-wrap align-items-center full-height"}).text.strip()
            lang = lang[:len(lang)//2]
            profiency = sample.find('span', {'class':"t-14 t-normal t-black--light"}).text.strip()
            profiency = profiency[:len(profiency)//2]

            return {'language':lang, 'profiency':profiency}

        langs = []
        for i in all_langs:
            langs.append(extract_langs(i))
        
        if langs == []:
            langs = get_languages()
        return langs
    except:
        return get_languages
    
if 'languages' in sections_available:
    languages = get_languages()
else:
    languages = [{
        'language': None,
        'profiency': None
    }]

# Honors and Awards

In [107]:
def get_awards():
    try:
        driver.get(profile_link + 'details/honors/')
        wait = WebDriverWait(driver, 10)
        soup = create_soup(driver.page_source)

        all_awards = soup.find_all('li', {'class', 'pvs-list__paged-list-item artdeco-list__item pvs-list__item--line-separated pvs-list__item--one-column'})

        def extract_langs(sample):
            '''
            award_name, org, date
            '''

            award_name = sample.find('div', {'class':"display-flex flex-wrap align-items-center full-height"}).text.strip()
            award_name = award_name[:len(award_name)//2]
            org_and_date = sample.find('span', {'class':"t-14 t-normal"}).text.strip()
            org_and_date = org_and_date[:len(org_and_date)//2].split(' · ')
            org = org_and_date[0]
            date = datetime.strptime(org_and_date[1], '%b %Y')
            return {'award_name':award_name, 'org':org, 'date':date}


        awards = []
        for i in all_awards:
            awards.append(extract_langs(i))
        
        if awards == []:
            awards = get_awards()
        return awards
    except:
        return get_awards

if 'honors_and_awards' in sections_available:
    awards = get_awards()
else:
    awards = [{
        'award_name': None,
        'org': None,
        'date': None
    }]